# https://www.kaggle.com/c/digit-recognizer/
# with Tensorflow, the CPU mode took too much time on KNN.

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.decomposition import PCA
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [3]:
config = tf.ConfigProto(device_count={"CPU": 12}, # limit to num_cpu_core CPU usage
                inter_op_parallelism_threads = 1, 
                intra_op_parallelism_threads = 4,
                log_device_placement=True)

In [4]:
# load csv files to numpy arrays

def load_data():
    train = pd.read_csv("train.csv")
    print(train.shape)
    X_train = train.values[:,1:] 
    y_train = train.values[:,0]
    
    
    Pred_test = pd.read_csv("test.csv").values
    return X_train, y_train, Pred_test

x_train, y_train, pred_test = load_data()

(42000, 785)


In [5]:
print(x_train.shape, y_train.shape, pred_test.shape)

(42000, 784) (42000,) (28000, 784)


In [6]:
#x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, train_size=0.8, random_state=0)
#print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

In [7]:
'''pca = PCA(n_components=50)
pca.fit(x_train)
x_train = pca.transform(x_train)
x_test = pca.transform(x_test)'''

'pca = PCA(n_components=50)\npca.fit(x_train)\nx_train = pca.transform(x_train)\nx_test = pca.transform(x_test)'

In [8]:
#sess = tf.InteractiveSession(config=config)
#sess = tf.InteractiveSession()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [9]:
'''
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
#def get_prediction(distance, k):
# Get k-min distance index (Nearest neighbor)
top_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)
prediction_indices = tf.gather(y_train, top_k_indices)

y, idx, cnts = tf.unique_with_counts(prediction_indices,)
max_index = tf.argmax(cnts) #vote it from k

# Predict the mode category
#    return y[max_index]
'''

'\ntop_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)\n#def get_prediction(distance, k):\n# Get k-min distance index (Nearest neighbor)\ntop_k_xvals, top_k_indices = tf.nn.top_k(tf.negative(distance), k=k)\nprediction_indices = tf.gather(y_train, top_k_indices)\n\ny, idx, cnts = tf.unique_with_counts(prediction_indices,)\nmax_index = tf.argmax(cnts) #vote it from k\n\n# Predict the mode category\n#    return y[max_index]\n'

In [10]:
%%time
k = 5
batch_size=30

# Placeholders
x_data_train = tf.placeholder(shape=x_train.shape, dtype=tf.float32)
x_data_test = tf.placeholder(shape=[None, 784], dtype=tf.float32)
y_target_train = tf.placeholder(shape=y_train.shape, dtype=tf.float32)
y_target_test = tf.placeholder(shape=[batch_size], dtype=tf.float32)

distance = tf.reduce_sum(
    tf.abs(
        tf.subtract(x_data_train, tf.expand_dims(x_data_test, 1))
    ), axis=2)

# Prediction: Get min distance index (Nearest neighbor)
pred = tf.argmin(distance, 1)

prediction = tf.gather(y_target_train, pred)
#correct_prediction = tf.equal(prediction, y_target_test)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Calculate how many loops over training data
num_loops = int(np.ceil(len(pred_test)/batch_size))

test_output = []
actual_vals = []
for i in range(num_loops):
    min_index = i*batch_size
    max_index = min((i+1)*batch_size,len(x_train))
    print(max_index)
    
    x_batch = pred_test[min_index:max_index]
    #x_batch = x_test[min_index:max_index]
    #y_batch = y_test[min_index:max_index]
    predictions = sess.run(prediction, feed_dict={x_data_train: x_train, 
                                                  x_data_test: x_batch,
                                                  y_target_train: y_train
                                                  #y_target_test: y_batch
                                                 })
    test_output.extend(predictions)
    #actual_vals.extend(y_batch)




#correct_prediction = tf.equal(test_output, actual_vals)
#accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
#print('Accuracy on test set: ' + str(accuracy))

predictions = np.array(test_output).astype(int)

def flush_predictons(predictions):
    imgs = pd.Series(range(1,pred_test.shape[0]+1))
    imgd = pd.DataFrame(imgs)
    
    labeld = pd.DataFrame(predictions)
    out = pd.concat([imgd, labeld],axis=1)
    out.columns = ["ImageId", "Label"]
    out.to_csv("KNN_PCA_Prediction.csv", index = False)

flush_predictons(predictions);

30
60
90
120
150
180
210
240
270
300
330
360
390
420
450
480
510
540
570
600
630
660
690
720
750
780
810
840
870
900
930
960
990
1020
1050
1080
1110
1140
1170
1200
1230
1260
1290
1320
1350
1380
1410
1440
1470
1500
1530
1560
1590
1620
1650
1680
1710
1740
1770
1800
1830
1860
1890
1920
1950
1980
2010
2040
2070
2100
2130
2160
2190
2220
2250
2280
2310
2340
2370
2400
2430
2460
2490
2520
2550
2580
2610
2640
2670
2700
2730
2760
2790
2820
2850
2880
2910
2940
2970
3000
3030
3060
3090
3120
3150
3180
3210
3240
3270
3300
3330
3360
3390
3420
3450
3480
3510
3540
3570
3600
3630
3660
3690
3720
3750
3780
3810
3840
3870
3900
3930
3960
3990
4020
4050
4080
4110
4140
4170
4200
4230
4260
4290
4320
4350
4380
4410
4440
4470
4500
4530
4560
4590
4620
4650
4680
4710
4740
4770
4800
4830
4860
4890
4920
4950
4980
5010
5040
5070
5100
5130
5160
5190
5220
5250
5280
5310
5340
5370
5400
5430
5460
5490
5520
5550
5580
5610
5640
5670
5700
5730
5760
5790
5820
5850
5880
5910
5940
5970
6000
6030
6060
6090
6120
6150
6180
6210
6

### Wall time: 1h 8min 57s